<a href="https://colab.research.google.com/github/HarmanHans/Twitter-Crawler/blob/main/geosearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [ ]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive', force_remount = True)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [29]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/washingtontweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "rXDGQzJFskLrRRKNrXUG6xEB7"
    consumer_secret = "CuCPM2RJGieynRAgguBbcpRMdINmsZBVXyAbvTpZEJ8X0PtQQ8"
    access_token = "1519042779027951616-p51C7lOvlRtxsL3DB6LVjSspJlzDFQ"
    access_token_secret = "mS7xk9PY9ltWxHCFEcq6sgNGIdlPlhWqWTMgUXRR4pnng"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-123.951187, 45.590307, -117.032391, 48.999172]  # Washington

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1533653514546622464, 'username': 'deepakmaini', 'created_at': 'Mon Jun 06 03:34:26 +0000 2022', 'lng': -122.3306025, 'lat': 47.614817, 'text': 'It’s hard to beat any leader who combines great resolve, persistence, mission focus, and clarity about what is and… https://t.co/zQQQyFhGc2'}
{'id': 1533653536759611392, 'username': 'rrnunez', 'created_at': 'Mon Jun 06 03:34:31 +0000 2022', 'lng': -122.2800145, 'lat': 47.469552, 'text': '@elonmusk @mlakan @EpochTimes When someone cites this "source" there\'s something strange in their form of thinking.… https://t.co/dAoTcracbU'}
{'id': 1533653571241009153, 'username': 'Luke_The_Duke00', 'created_at': 'Mon Jun 06 03:34:39 +0000 2022', 'lng': -122.630908, 'lat': 45.536402, 'text': 'Proud of myself today lmao'}
{'id': 1533653572302143488, 'username': 'pascha_scott', 'created_at': 'Mon Jun 06 03:34:40 +0000 2022', 'lng': -121.966606, 'lat': 47.7363335, 'text': 'Happy #WorldEnvironmentDay'}
{'id': 1533653583840694272, 'username': 'Smedindy', 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
